In [1]:
import os
from tvb.simulator.lab import *
import numpy as np
import seaborn
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io
LOG = get_logger('demo')
import pickle as cPickle
from tvb.simulator.models.stefanescu_jirsa import ReducedSetHindmarshRose
from os.path import join as pjoin

/Users/yilewang/miniconda3/envs/tvbenv/lib/python3.11/site-packages/tvb/datatypes/surfaces.py:60: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


### General workflow of data analyses

1. load fmri simulation
2. calculate the fc then calculate the correlation between simulated fc and empirical fc
3. save the correlation coefficient then plot the correlation trajectory for each participant
4. visualize the results

In [ ]:
base_dir = "/Users/yilewang/Desktop/"
data_dir = pjoin(base_dir, "local_inhibition")

eFC_dir = "/Users/yilewang/workspaces/data4project/lateralization/ts_fmri/"

def FC_fitting(group, caseid, K21, dir):
    subj_dir = pjoin(dir, group, caseid+'.csv')
    s_ts = pd.read_csv(subj_dir)
    # calculate the simulated FC
    s_fc = s_ts.corr()
    # compare it to the empirical FC
    mat = scipy.io.loadmat(pjoin(eFC_dir, group+"-TS", "ROISignals_"+caseid+'.mat'))
    e_ts = mat['ROISignals']
    e_fc = e_ts.corr()
    # vectorize the lower triangle of the matrix
    s_fc_vec = np.tril(s_fc, k=-1).reshape(-1)
    e_fc_vec = np.tril(e_fc, k=-1).reshape(-1)
    # calculate the correlation coefficient
    corr = np.corrcoef(s_fc_vec, e_fc_vec)[0,1]
# iterate then I can have a plot for inidividuals.